In [1]:
# Do this only in Colab notebooks! Otherwise use pip install unsloth
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo -q
!pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer -q
!pip install --no-deps unsloth -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 43.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 30.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 10.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.8.9 requires msgspec, which is not installed.
unsloth-zoo 2025.8.9 requires tyro, which is not installed.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 

In [2]:
import torch
import gc
import json
import os
from PIL import Image
from unsloth import FastVisionModel
from typing import Union, Dict, List, Optional
import re
from tqdm import tqdm
import pandas as pd
import time

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-08-31 14:53:58.914509: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756652039.244777      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756652039.339965      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# ===================================================================
# Block 1: Data Loading and Utility Functions (MODIFIED FOR TRAIN SET)
# ===================================================================

import json
import os
import re
from PIL import Image

def load_train_data() -> tuple:
    """
    Load all training data, law DB, and pre-extracted article details.
    The 'answer' is already included in the training JSON.
    """
    train_json_path = "/kaggle/input/vlsp-dataset/VLSP 2025 - MLQA-TSR Data Release-20250820T023346Z-1-001/VLSP 2025 - MLQA-TSR Data Release/train_data/vlsp_2025_train.json"
    law_db_path = "/kaggle/input/vlsp-dataset/VLSP 2025 - MLQA-TSR Data Release-20250820T023346Z-1-001/VLSP 2025 - MLQA-TSR Data Release/law_db/vlsp2025_law_new.json"
    extracted_data_path = "/kaggle/input/vlsp-summarize-final/train_vision_rule_extraction_results.csv"

    print(f"Loading training data from: {train_json_path}")
    print(f"Loading law database from: {law_db_path}")
    print(f"Loading extracted article data from: {extracted_data_path}")

    with open(train_json_path, 'r', encoding='utf-8') as f:
        # Training data already contains the answer key for all items
        train_questions = json.load(f)
    with open(law_db_path, 'r', encoding='utf-8') as f:
        law_database = json.load(f)

    extracted_data_df = pd.read_csv(extracted_data_path)

    print(f"Loaded {len(train_questions)} total training questions.")
    print(f"Loaded {len(law_database)} laws in database.")
    print(f"Loaded {len(extracted_data_df)} rows from extracted data CSV.")
    
    return train_questions, law_database, extracted_data_df


def get_relevant_articles(question_data: dict, law_database: list) -> list:
    """
    Extract relevant articles for a given question from the raw law database.
    """
    relevant_articles = []
    for ref_article in question_data.get("relevant_articles", []):
        law_id, article_id = ref_article["law_id"], ref_article["article_id"]
        for law in law_database:
            if law["id"] == law_id:
                for article in law["articles"]:
                    if article["id"] == article_id:
                        relevant_articles.append({
                            "law_id": law_id, "law_title": law["title"], "article_id": article["id"],
                            "article_title": article["title"], "article_text": article["text"]
                        })
                        break
                break
    return relevant_articles


def clean_article_text(text: str) -> str:
    """
    Clean article text by removing image and table tags.
    """
    clean_text = re.sub(r'<<IMAGE: (.*?) /IMAGE>>', '', text)
    return re.sub(r'<<TABLE: (.*?) /TABLE>>', '', clean_text, flags=re.DOTALL).strip()


def resize_image_if_needed(image_path: str, threshold: int = 768) -> Image.Image:
    """
    Loads an image and resizes it proportionally if either dimension exceeds the threshold.
    """
    img = Image.open(image_path).convert("RGB")
    return img

In [4]:
# ===================================================================
# Block 2: Prompt Generation and Processing (ADJUSTED FOR NEW EXTRACTION)
# ===================================================================

def make_prompt(
    question_id: str,
    question: str,
    choices: Optional[Dict[str, str]],
    question_type: str,
    relevant_articles: List[dict], # Used only for the fallback mechanism
    extracted_data_df: pd.DataFrame
) -> str:
    """
    Creates a formatted CoT prompt using pre-extracted data (one entry per question)
    with a fallback to the raw law DB.
    """
    prompt_parts = ["Bạn là một chuyên gia về luật giao thông Việt Nam. Hãy trả lời câu hỏi sau dựa trên các điều luật được cung cấp.\n"]

    if question_type == "Multiple choice":
        extend_2 = "Phân tích luật, câu hỏi và lần lượt phân tích cả A, B, C và D. Giải thích tại sao câu trả lời cuối cùng là đúng, và các câu trả lời còn lại là sai"
        step_4 = "Step 3: [Duyệt qua và phân tích cả A, B, C và D.]"
    else: # Yes/No
        extend_2 = "Phân tích luật và câu hỏi, phân tích cho cả hai hướng là câu trả lời sai và câu trả lời đúng."
        step_4 = "Step 3: [Chú ý xác định đây là mệnh đề khẳng định hay phủ định.]"
        
    prompt_parts.extend([
        "\n# INSTRUCTION",
        "Hãy suy nghĩ từng bước và trả lời theo định dạng yêu cầu:",
        extend_2,
    ])
    
    question_extraction = extracted_data_df[extracted_data_df['question_id'] == question_id]

    prompt_parts.append("\n# BIỂN BÁO TRONG ẢNH:")
    
    use_fallback = True
    if not question_extraction.empty:
        row = question_extraction.iloc[0]
        # The training extraction script uses 'raw' which contains all details
        raw_extraction = row.get('raw')
        if pd.notna(raw_extraction) and "error" not in str(raw_extraction).lower():
            prompt_parts.append(raw_extraction)
            use_fallback = False

    # --- FALLBACK LOGIC ---
    if use_fallback:
        print(f"  -> NOTE: Using fallback for question {question_id}. No valid extraction found.")
        article_count = 0
        
        for article in relevant_articles:
            article_text = clean_article_text(article['article_text'])
            text_snippet = " ".join(article_text.split()[:100])
            if len(article_text.split()) > 100:
                text_snippet += "..."

            prompt_parts.extend([
                f"## Tên: {article['article_title']}",
                f"Nội dung: {text_snippet}",
                ""
            ])
            article_count += 1
        
        if article_count == 0: 
            prompt_parts.append("Không tìm thấy thông tin luật liên quan.")

    general_knowledge = """# EXCEPTION
Không chịu tác động của biển báo: xe cấp cứu, xe cảnh sát, xe ưu tiên."""
    prompt_parts.append(general_knowledge)
        
    prompt_parts.extend([
        "\n# ĐỊNH DẠNG OUTPUT",
        "Cung cấp câu trả lời cuối cùng theo định dạng sau, không thêm bất kỳ văn bản nào khác:",
        "<reasoning>",
        "Step 1: [Phân tích câu hỏi]",
        "Step 2: [Phân tích # BIỂN BÁO TRONG ẢNH]",
        step_4,
        "</reasoning>",
    ])

    if question_type == "Multiple choice":
        prompt_parts.append("<answer>Chỉ một chữ cái (A, B, C, hoặc D)</answer>")
    else: # Yes/No
        prompt_parts.append("<answer>Chỉ 'True' hoặc 'False'</answer>")

    prompt_parts.extend(["# CÂU HỎI:", f"{question}\n"])

    if question_type == "Multiple choice" and choices:
        prompt_parts.append("# CÁC LỰA CHỌN:")
        for key, value in choices.items():
            prompt_parts.append(f"<{key}>{value}</{key}>")

    prompt_parts.append("# OUTPUT")
    return "\n".join(prompt_parts)


def get_train_image_path(image_id: str) -> str:
    """
    Get the full path to a training image.
    """
    base_path = "/kaggle/input/vlsp-dataset/VLSP 2025 - MLQA-TSR Data Release-20250820T023346Z-1-001/VLSP 2025 - MLQA-TSR Data Release/train_data/train_images/train_images"
    image_filename = f"{image_id}.jpg"
    
    image_path = os.path.join(base_path, image_filename)
    if not os.path.exists(image_path):
        image_path = image_path.replace(".jpg", ".png")
    return image_path


def process_train_question(question_data: dict, law_database: list, extracted_data_df: pd.DataFrame) -> tuple:
    """
    Process a single training question to get prompt, image path, and answer.
    """
    prompt = make_prompt(
        question_id=question_data["id"],
        question=question_data["question"],
        choices=question_data.get("choices"),
        question_type=question_data["question_type"],
        relevant_articles=get_relevant_articles(question_data, law_database),
        extracted_data_df=extracted_data_df
    )
    image_path = get_train_image_path(question_data["image_id"])
    return prompt, image_path, question_data.get("answer"), question_data["id"]

In [5]:
# ===================================================================
# Block 3: Inference Class (UNCHANGED)
# ===================================================================

class SimpleVisionInference:
    """Simple 2-GPU vision model inference using Unsloth."""
    def __init__(self, model_name: str = "unsloth/Qwen2.5-VL-7B-Instruct-bnb-4bit"):
        self.model, self.processor = None, None
        self.model_name = model_name
        print(f"Available GPUs: {torch.cuda.device_count()}")
        device_map = "balanced" if torch.cuda.device_count() >= 2 else "auto"
        print(f"Loading model from: {model_name} with device_map: {device_map}")
        self._load_model(device_map)
        print("Model loaded successfully!")

    def _load_model(self, device_map: str):
        try:
            self.model, self.processor = FastVisionModel.from_pretrained(
                self.model_name, load_in_4bit=True, device_map=device_map
            )
            FastVisionModel.for_inference(self.model)
        except Exception as e:
            raise RuntimeError(f"Failed to load model {self.model_name}: {str(e)}")

    def inference(self, prompt: str, image_path: str, max_new_tokens: int = 8, **kwargs) -> str:
        """
        Perform inference, now with integrated resizing and logging.
        """
        try:
            pil_image = resize_image_if_needed(image_path, threshold=768)
            print(f"\nLOG: Processing Image '{os.path.basename(image_path)}'")
            print(f"  Image Size (for Inference): {pil_image.size} | Prompt Chars: {len(prompt)}")
            device = next(self.model.parameters()).device
            messages = [{"role": "user", "content": [{"type": "image"}, {"type": "text", "text": prompt}]}]
            text = self.processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            inputs = self.processor(text=[text], images=[pil_image], return_tensors="pt").to(device)
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    use_cache=True,
                    temperature=0.1
                )
            return self.processor.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)[0].strip()
        except Exception as e:
            raise RuntimeError(f"Inference failed: {str(e)}")

    def cleanup(self):
        """Clean up VRAM."""
        print("🧹 Cleaning up VRAM...")
        del self.model
        del self.processor
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        print("VRAM cleaned up")

    def __del__(self):
        self.cleanup()

In [6]:
# ===================================================================
# Block 4: Full Dataset Inference Workflow (MODIFIED FOR TRAIN SET)
# ===================================================================

def parse_cot_output(raw_output: str) -> (str, str):
    """
    Parses the CoT output to extract reasoning and the final answer.
    """
    reasoning_match = re.search(r"<reasoning>(.*?)</reasoning>", raw_output, re.DOTALL)
    answer_match = re.search(r"<answer>(.*?)</answer>", raw_output, re.DOTALL)
    reasoning = reasoning_match.group(1).strip() if reasoning_match else "No reasoning found."
    answer = answer_match.group(1).strip() if answer_match else "No answer found."
    if(reasoning == "No reasoning found."):
        print(raw_output)
    return reasoning, answer

def inference_all_questions(model_instance: SimpleVisionInference, questions: list, law_database: list, extracted_data_df: pd.DataFrame, output_csv: str):
    """
    Run CoT inference on all training questions, evaluate, log periodically, and save results.
    """
    results = []
    correct_count, total_processed = 0, 0
    start_time = time.time()
    answer_map = {"Yes": "True", "No": "False"}

    PROMPT_SANITY_CHECK_COUNT = 3

    if not questions:
        print("No questions to process.")
        return
    print(f"\n🔄 Running inference on a batch of {len(questions)} questions (from ID '{questions[0]['id']}' to '{questions[-1]['id']}')...")
    
    for index, question_data in enumerate(tqdm(questions, desc="Processing Questions")):
        result_record = {
            'question_id': question_data['id'],
            'correct_answer': question_data.get('answer'),
            'reasoning': '',
            'model_answer': '',
            'is_correct': False
        }
        
        try:
            prompt, image_path, correct_answer, _ = process_train_question(
                question_data, law_database, extracted_data_df
            )
            
            if index < PROMPT_SANITY_CHECK_COUNT:
                print(f"\n--- PROMPT SANITY CHECK (Question: {question_data['id']}) ---")
                print(prompt)
                print("--------------------------------------------------")

            if os.path.exists(image_path):
                raw_model_output = model_instance.inference(prompt, image_path, max_new_tokens=1024)
                reasoning, final_answer = parse_cot_output(raw_model_output)
                
                if index < PROMPT_SANITY_CHECK_COUNT:
                    print(f"\n--- Sanity Check (Question: {question_data['id']}) ---")
                    print(f"  Reasoning: {reasoning}")
                    print(f"  Model Answer: {final_answer} | Correct Answer: {correct_answer}")
                    print("-------------------------------------------------")

                is_correct = False
                if question_data['question_type'] == "Yes/No":
                    if final_answer == answer_map.get(correct_answer):
                        is_correct = True
                else: # Handles "Multiple choice"
                    if final_answer == correct_answer:
                        is_correct = True
                
                if is_correct:
                    correct_count += 1

                result_record.update({
                    'reasoning': reasoning,
                    'model_answer': final_answer,
                    'is_correct': is_correct
                })
                total_processed += 1
            else:
                result_record['model_answer'] = "ERROR: Image not found"
                print(f"  Skipping {question_data['id']}: Image not found at {image_path}")
                
        except Exception as e:
            result_record['model_answer'] = f"ERROR: {e}"
            print(f"\n  ❌ Error processing question {question_data['id']}: {e}")
        
        results.append(result_record)

    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False, encoding='utf-8')
    total_time = time.time() - start_time
    accuracy = (correct_count / total_processed * 100) if total_processed > 0 else 0
    
    print("\n" + "="*50 + "\n📊 INFERENCE SUMMARY")
    print(f"Total Questions Processed: {total_processed}, Correct Guesses: {correct_count}")
    print(f"Accuracy: {accuracy:.2f}%")
    if total_time > 0:
        print(f"Images per Second: {total_processed / total_time:.2f} img/s")
    print(f"Results for this batch saved to: {output_csv}")

In [7]:
# --- MODIFIED: Main function now handles the training data ---
def main():
    """
    Main function for running inference on the training set.
    """
    print("🚀 Vietnamese Law QA System - Training Set Inference")
    print("=" * 70)

    output_csv = "train_inference_results.csv"
    model_instance = None
    try:
        print("📚 Loading Vietnamese Law Training Dataset and Extracted Data...")
        train_questions, law_database, extracted_data_df = load_train_data()

        print("\n🤖 Initializing Qwen2.5-VL Model...")
        model_instance = SimpleVisionInference()

        inference_all_questions(
            model_instance=model_instance,
            questions=train_questions[:25], # Using a slice for initial testing
            law_database=law_database,
            extracted_data_df=extracted_data_df,
            output_csv=output_csv
        )

        print(f"\n🎉 Inference completed! Check {output_csv} for detailed results.")

    except Exception as e:
        print(f"❌ A critical error occurred: {e}")
        import traceback
        traceback.print_exc()

    finally:
        if model_instance:
            del model_instance

if __name__ == "__main__":
    main()

🚀 Vietnamese Law QA System - Training Set Inference
📚 Loading Vietnamese Law Training Dataset and Extracted Data...
Loading training data from: /kaggle/input/vlsp-dataset/VLSP 2025 - MLQA-TSR Data Release-20250820T023346Z-1-001/VLSP 2025 - MLQA-TSR Data Release/train_data/vlsp_2025_train.json
Loading law database from: /kaggle/input/vlsp-dataset/VLSP 2025 - MLQA-TSR Data Release-20250820T023346Z-1-001/VLSP 2025 - MLQA-TSR Data Release/law_db/vlsp2025_law_new.json
Loading extracted article data from: /kaggle/input/vlsp-summarize-final/train_vision_rule_extraction_results.csv
Loaded 530 total training questions.
Loaded 2 laws in database.
Loaded 441 rows from extracted data CSV.

🤖 Initializing Qwen2.5-VL Model...
Available GPUs: 2
Loading model from: unsloth/Qwen2.5-VL-7B-Instruct-bnb-4bit with device_map: balanced
==((====))==  Unsloth 2025.8.10: Fast Qwen2_5_Vl patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    T

model.safetensors:   0%|          | 0.00/5.97G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

  -> NOTE: Using fallback for question train_24. No valid extraction found.

LOG: Processing Image 'train_1_9.jpg'
  Image Size (for Inference): (495, 341) | Prompt Chars: 1692


Processing Questions:  96%|█████████▌| 24/25 [15:50<00:35, 35.22s/it]

  -> NOTE: Using fallback for question train_25. No valid extraction found.

LOG: Processing Image 'train_1_20.jpg'
  Image Size (for Inference): (500, 334) | Prompt Chars: 1436


Processing Questions: 100%|██████████| 25/25 [16:23<00:00, 39.35s/it]



📊 INFERENCE SUMMARY
Total Questions Processed: 25, Correct Guesses: 15
Accuracy: 60.00%
Images per Second: 0.03 img/s
Results for this batch saved to: train_inference_results.csv

🎉 Inference completed! Check train_inference_results.csv for detailed results.
🧹 Cleaning up VRAM...
VRAM cleaned up


In [8]:
# ===================================================================
# Block 5: Post-Inference Analysis (NEW)
# ===================================================================
import pandas as pd

try:
    # Load the results from the training run
    output_csv = "train_inference_results.csv"
    df = pd.read_csv(output_csv)

    print(f"\n--- Analysis of {output_csv} ---")

    # Display a count of the model's answers vs correct answers
    print("\nModel answer distribution:")
    print(df['model_answer'].value_counts())
    print("\nCorrect answer distribution:")
    print(df['correct_answer'].value_counts())

    # The 'is_correct' column is already calculated during inference.
    # We can directly calculate the accuracy from it.
    if 'is_correct' in df.columns and not df['is_correct'].isnull().all():
        # Ensure boolean type for correct calculation, handling potential non-boolean values
        df['is_correct'] = df['is_correct'].astype(bool)
        accuracy = df['is_correct'].mean() * 100
        print(f"\n✅ Final Accuracy on the processed training set slice: {accuracy:.2f}%")
        
        # Display rows where the model was incorrect
        print("\n🔍 Examples of incorrect answers:")
        incorrect_df = df[df['is_correct'] == False]
        print(incorrect_df[['question_id', 'correct_answer', 'model_answer', 'reasoning']].head())

    else:
        print("\n'is_correct' column not found or is empty. Cannot calculate accuracy.")

except FileNotFoundError:
    print(f"\nCould not find the results file: {output_csv}. Please run the main inference block first.")


--- Analysis of train_inference_results.csv ---

Model answer distribution:
model_answer
A        11
False     5
B         4
C         3
D         1
True      1
Name: count, dtype: int64

Correct answer distribution:
correct_answer
A        9
B        6
Đúng    4
C        3
Sai      2
D        1
Name: count, dtype: int64

✅ Final Accuracy on the processed training set slice: 60.00%

🔍 Examples of incorrect answers:
  question_id correct_answer model_answer  \
0     train_1              B            A   
4     train_5              B            A   
7     train_8          Đúng        False   
8     train_9          Đúng        False   
9    train_10            Sai        False   

                                           reasoning  
0  Step 1: Phân tích câu hỏi\n- Câu hỏi yêu cầu t...  
4  Step 1: Phân tích câu hỏi: Câu hỏi yêu cầu phâ...  
7  Step 1: Phân tích câu hỏi\n- Câu hỏi yêu cầu x...  
8  Step 1: Phân tích câu hỏi\n- Câu hỏi yêu cầu x...  
9  Step 1: Phân tích câu hỏi\n- C